# Visual odometry with Kimera package

## Kimera-VIO installation

See

0. Kimera home - https://github.com/MIT-SPARK/Kimera

1. Kimera image preparation: https://github.com/MIT-SPARK/Kimera-VIO/blob/master/docs/kimera_vio_install.md

2. Udocker for running containers in Colab - https://github.com/indigo-dc/udocker/blob/master/docs/installation_manual.md and https://gist.github.com/mwufi/6718b30761cd109f9aff04c5144eb885

3. Test data origin - https://drive.google.com/drive/folders/1-B6V-4S_ZsCqjZY5muXNxV_dqbmdeAto

4. Downloading files from Google Drive - https://colab.research.google.com/notebooks/io.ipynb

5. Downloading files [from ydisk](https://ru.stackoverflow.com/questions/1088300/%D0%BA%D0%B0%D0%BA-%D1%81%D0%BA%D0%B0%D1%87%D0%B8%D0%B2%D0%B0%D1%82%D1%8C-%D1%84%D0%B0%D0%B9%D0%BB%D1%8B-%D1%81-%D1%8F%D0%BD%D0%B4%D0%B5%D0%BA%D1%81-%D0%B4%D0%B8%D1%81%D0%BA%D0%B0).

In [ ]:
# !udocker --allow-root run docker.io/zaba247/kimera-vio ls # run -p 127.0.0.1:8081:8081 docker.io/zaba247/kimera-vio:latest ls

In [ ]:
import os
if not os.path.exists("/home/user"):
    !useradd -m user
!pip install udocker
!udocker --allow-root install

In [ ]:
import requests
from urllib.parse import urlencode


def download(url: str, output: str):
    base_url = "https://cloud-api.yandex.net/v1/disk/public/resources/download?"
    final_url = base_url + urlencode(dict(public_key=url))
    response = requests.get(final_url)
    download_url = response.json()['href']

    download_response = requests.get(download_url)
    with open(output, "wb") as f:
        f.write(download_response.content)

In [ ]:
download(url="https://disk.yandex.ru/d/Rd5d7BtgIFE3OQ", output="mh01.zip.001")
download(url="https://disk.yandex.ru/d/J6ryB7H8v-u1Ig", output="mh01.zip.002")

In [ ]:
!cat mh01.zip.* > ./mh01.zip

In [ ]:
!unzip -o ./mh01.zip > /dev/null

In [ ]:
!su - user -c "udocker pull docker.io/zaba247/kimera-vio"

In [ ]:
# !su - user -c "udocker create --help"

In [ ]:
!su - user -c "udocker create --force --name=kimera docker.io/zaba247/kimera-vio"

In [ ]:
# !git clone https://github.com/MIT-SPARK/Kimera-VIO.git

In [ ]:
# !mkdir ./scripts/

In [ ]:
!su - user -c "udocker --allow-root run -v '"`pwd`"/scripts/:/scripts' -v '"`pwd`"/Kimera-VIO/:/kimera-vio' -v '"`pwd`"/MH_01_easy/mav0:/mav0' kimera /root/Kimera-VIO/scripts/stereoVIOEuroc.bash"

## Building Kimera from source (deprecated)

In [ ]:
raise SystemExit("Stop right there!")

In [ ]:
!sudo apt-get update
!sudo apt-get install -y --no-install-recommends apt-utils
!sudo apt-get install -y cmake
!sudo apt-get install -y libboost-all-dev

In [ ]:
!sudo apt-get install -y \
      build-essential unzip pkg-config \
      libjpeg-dev libpng-dev libtiff-dev \
      libvtk7-dev \
      libgtk-3-dev \
      libparmetis-dev \
      libatlas-base-dev gfortran

In [ ]:
!sudo apt-get install libtbb-dev

In [ ]:
!git clone https://github.com/borglab/gtsam.git

In [ ]:
!cd gtsam && git checkout 4.2 && mkdir build && cd build && cmake -DCMAKE_INSTALL_PREFIX=/usr/local -DCMAKE_BUILD_TYPE=Release -DGTSAM_USE_SYSTEM_EIGEN=ON -DGTSAM_POSE3_EXPMAP=ON -DGTSAM_ROT3_EXPMAP=ON -DGTSAM_TANGENT_PREINTEGRATION=OFF ..

In [ ]:
!cd gtsam/build && make -j $(nproc) check && sudo make -j $(nproc) install

## Downloading files from camera

In [ ]:
file_id = "1BgTnHHsRwYFGRBRVxW_bj1pr0or62YOX"

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))

In [ ]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')